In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
eps = []
titles = []
text = []

In [ ]:
# parses transcript links
class Lore(scrapy.Spider):
    name = 'Lore'
    
    def start_requests(self):
        base_url = 'https://www.nutritiousmovement.com/category/podcast-transcripts/page/'
        urls = []
        for i in range(2, 15):
            urls.append(base_url + str(i) + '/')
        # print(urls) # Urls are correct
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_episode)
        
    def parse_episode(self, response):
        transcript_links = response.xpath('//main[@class="content"]//article/div/header/h2/a/@href').extract() 
        # print([l for l in transcript_links])
        for link in transcript_links:
            yield response.follow(url=link, 
                                  callback =self.parse_transcript)
    
    def parse_transcript(self, response):        
        # Title - done, clean
        title = str(response.xpath('//h1[@itemprop="headline"]/text()').extract())
        clean_title = title[2:-2].split(' – Podcast Episode #')
        ep_title = clean_title[0]
        ep_number = clean_title[1]
        
        # Text - NEEDS CLEANED!
        text = response.css(' div.entry-content p > *::text').extract()
        text = [t.lower() for t in text]
        text = text[text.index('podcast transcript') + 1:]
        myDNA_data.append((ep_number, ep_title, text))